# 2 degree-of-freedom example of derivative-free optimization involving VMEC

This script implements the "2DOF_vmecOnly_targetIotaAndVolume" example from https://github.com/landreman/stellopt_scenarios . This optimization problem has two independent variables, representing the helical shape of the magnetic axis. The problem also has two objectives: the plasma volume and the rotational transform on the magnetic axis.

The resolution in this example (i.e. ns, mpol, and ntor) is somewhat lower than in the stellopt_scenarios version of the example, just so this example runs fast.

Details of the optimum and a plot of the objective function landscape can be found here: https://github.com/landreman/stellopt_scenarios/tree/master/2DOF_vmecOnly_targetIotaAndVolume  

In [2]:
import sys
sys.path.append('..')
import numpy as np
from simsopt import Vmec, LeastSquaresProblem, least_squares_serial_solve

Initialize VMEC and its boundary surface shape from an input file:

In [3]:
equil = Vmec('input.2DOF_vmecOnly_targetIotaAndVolume')
surf = equil.boundary

Now determine which parameters are varied in the optimization. VMEC parameters are all fixed by default, while surface parameters are all non-fixed by default. You can choose which parameters are optimized by setting their 'fixed' attributes.

In [4]:
surf.all_fixed()
surf.set_fixed('rc(1,1)', False)
surf.set_fixed('zs(1,1)', False)

Each function you want to optimize is then equipped with a shift and weight, to become a term in a least-squares objective function. Each term can be a tuple or list:

In [5]:
desired_volume = 0.15
volume_weight = 1
term1 = (equil.volume, desired_volume, volume_weight)

desired_iota = 0.41
iota_weight = 1
term2 = (equil.iota_axis, desired_iota, iota_weight)

A list or tuple of terms are combined to form a nonlinear-least-squares problem:

In [6]:
prob = LeastSquaresProblem([term1, term2])

Let's print out the initial global state vector, i.e. the vector of variables that is optimized. Each entry in this state vector has an associated string, explaining its meaning.

In [7]:
print(prob.x)
print(prob.dofs.names)

[ 0.05 -0.05]
['rc(1,1) of SurfaceRZFourier 0x10c8c58b0 (nfp=5, stelsym=True, mpol=3, ntor=3)', 'zs(1,1) of SurfaceRZFourier 0x10c8c58b0 (nfp=5, stelsym=True, mpol=3, ntor=3)']


Simsopt detects that gradient information is not available:

In [8]:
prob.dofs.grad_avail

False

Finally, let's solve the optimization problem. Simsopt detects that analytic derivatives are not available, and so chooses a derivative-free algorithm. During the optimization, VMEC's output is printed to the terminal window running the Jupyter server, not directly in this notebook. Here we use the serial solver `least_squares_serial_solve` to avoid the complication of using jupyter with MPI, but an MPI solver using parallelized finite-difference gradients is also available.

In [9]:
least_squares_serial_solve(prob)

Using derivative-free method
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.3543e-01                                    8.22e+00    
       1              2         4.2763e-02      9.27e-02       7.07e-02       6.96e+00    
       2              3         2.1342e-02      2.14e-02       4.81e-02       3.10e+00    
       3              5         6.6945e-04      2.07e-02       1.72e-02       2.88e-01    
       4              7         4.0908e-04      2.60e-04       8.62e-03       3.74e-02    
       5              9         4.0003e-04      9.05e-06       4.31e-03       8.87e-03    
       6             11         3.9992e-04      1.11e-07       1.08e-03       9.71e-04    
       7             12         3.9981e-04      1.05e-07       2.69e-04       2.85e-04    
       8             13         3.9973e-04      8.25e-08       5.39e-04       6.92e-05    
       9             16         3.9973e-04      5.76e-10     

Let's examine the optimum:

In [10]:
print("At the optimum,")
print(" rc(m=1,n=1) = ", surf.get_rc(1, 1))
print(" zs(m=1,n=1) = ", surf.get_zs(1, 1))
print(" volume, according to VMEC    = ", equil.volume())
print(" volume, according to Surface = ", surf.volume())
print(" iota on axis = ", equil.iota_axis())
print(" objective function = ", prob.objective())

At the optimum,
 rc(m=1,n=1) =  0.031152203932509667
 zs(m=1,n=1) =  -0.031149613082043764
 volume, according to VMEC    =  0.1782375723663839
 volume, according to Surface =  0.17823757236638396
 iota on axis =  0.41144628797201466
 objective function =  0.0007994522420447627


This solution matches the description in https://github.com/landreman/stellopt_scenarios/tree/master/2DOF_vmecOnly_targetIotaAndVolume . We can do some asserts to be sure:

In [11]:
assert np.abs(surf.get_rc(1, 1) - 0.0313066948) < 1.0e-3
assert np.abs(surf.get_zs(1, 1) - (-0.031232391)) < 1.0e-3
assert np.abs(equil.volume() - 0.178091) < 1.0e-3
assert np.abs(surf.volume()  - 0.178091) < 1.0e-3
assert np.abs(equil.iota_axis() - 0.4114567) < 1.0e-4
assert prob.objective() < 1.0e-2